In [ ]:
!pip -q install ultralytics opencv-python pyyaml


import os, glob, zipfile, shutil, yaml
import numpy as np
import cv2
import torch
from ultralytics import YOLO
from google.colab import files

print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())


print("📤 Please select your dataset .zip …")
uploaded = files.upload()
assert len(uploaded) == 1, "Upload exactly one ZIP file."
zip_name = list(uploaded.keys())[0]
print("Uploaded:", zip_name)


DATASET_DIR = "/content/dataset"
if os.path.exists(DATASET_DIR):
    shutil.rmtree(DATASET_DIR)

TMP_DIR = "/content/_unz_tmp"
if os.path.exists(TMP_DIR):
    shutil.rmtree(TMP_DIR)
os.makedirs(TMP_DIR, exist_ok=True)

with zipfile.ZipFile(zip_name, 'r') as zf:
    zf.extractall(TMP_DIR)


entries = [os.path.join(TMP_DIR, e) for e in os.listdir(TMP_DIR)]
top_dirs = [e for e in entries if os.path.isdir(e)]
src = top_dirs[0] if (len(top_dirs) == 1 and len(entries) == 1) else TMP_DIR

shutil.copytree(src, DATASET_DIR)
shutil.rmtree(TMP_DIR)
print(" Unzipped to:", DATASET_DIR)


DATA_YAML = None
for candidate in [
    os.path.join(DATASET_DIR, "data.yaml"),
    os.path.join(DATASET_DIR, "dataset.yaml"),
]:
    if os.path.exists(candidate):
        DATA_YAML = candidate
        break

if DATA_YAML is None:
    hits = glob.glob(os.path.join(DATASET_DIR, "**", "data.yaml"), recursive=True)
    assert hits, "data.yaml not found anywhere in the dataset."
    DATA_YAML = hits[0]

with open(DATA_YAML, "r") as f:
    data_cfg = yaml.safe_load(f)


train_images = os.path.join(DATASET_DIR, "train", "images")
valid_images = os.path.join(DATASET_DIR, "valid", "images")
val_images   = os.path.join(DATASET_DIR, "val",   "images")
test_images  = os.path.join(DATASET_DIR, "test",  "images")

if os.path.isdir(valid_images):
    data_cfg["train"] = os.path.abspath(train_images)
    data_cfg["val"]   = os.path.abspath(valid_images)
elif os.path.isdir(val_images):
    data_cfg["train"] = os.path.abspath(train_images)
    data_cfg["val"]   = os.path.abspath(val_images)
else:

    for k in ("train", "val", "valid"):
        if k in data_cfg and isinstance(data_cfg[k], str) and not data_cfg[k].startswith("/"):
            data_cfg[k] = os.path.abspath(os.path.join(DATASET_DIR, data_cfg[k]))

if os.path.isdir(test_images):
    data_cfg["test"] = os.path.abspath(test_images)


if "names" in data_cfg and isinstance(data_cfg["names"], (list, tuple)):
    data_cfg["nc"] = len(data_cfg["names"])

with open(DATA_YAML, "w") as f:
    yaml.safe_dump(data_cfg, f, sort_keys=False)

print("Patched", DATA_YAML)
print(yaml.safe_dump(data_cfg, sort_keys=False))


def count_imgs(p):
    return len(glob.glob(os.path.join(p, "*.jpg"))) + len(glob.glob(os.path.join(p, "*.png")))

ti = count_imgs(os.path.join(DATASET_DIR, "train", "images"))
vi = count_imgs(os.path.join(DATASET_DIR, "valid", "images")) + \
     count_imgs(os.path.join(DATASET_DIR, "val", "images"))

print(f" Train images (before oversampling): {ti}")
print(f" Valid/Val images: {vi}")

for p in [
    os.path.join(DATASET_DIR, "train", "images"),
    os.path.join(DATASET_DIR, "train", "labels"),
    os.path.join(DATASET_DIR, "valid", "images"),
    os.path.join(DATASET_DIR, "valid", "labels"),
]:
    if os.path.isdir(p):
        some = sorted(glob.glob(os.path.join(p, "*.*")))[:5]
        print(" ", p, "->", len(some), "sample(s):", [os.path.basename(s) for s in some])


train_img_dir = os.path.join(DATASET_DIR, "train", "images")
train_lbl_dir = os.path.join(DATASET_DIR, "train", "labels")

label_files = sorted(glob.glob(os.path.join(train_lbl_dir, "*.txt")))
assert label_files, "No label files found in train/labels for oversampling."

class_counts = {}
img_classes = {}

for lbl_path in label_files:
    stem = os.path.splitext(os.path.basename(lbl_path))[0]
    img_classes.setdefault(stem, set())

    with open(lbl_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            cls_id = int(parts[0])
            class_counts[cls_id] = class_counts.get(cls_id, 0) + 1
            img_classes[stem].add(cls_id)

print("  Instance count per class (before oversampling):")
for cid in sorted(class_counts.keys()):
    print(f"  Class {cid}: {class_counts[cid]}")

max_count = max(class_counts.values()) if class_counts else 0
print("  Max instance count among classes:", max_count)

def find_image_for_stem(stem):
    exts = [".jpg", ".jpeg", ".png", ".bmp"]
    for ext in exts:
        path = os.path.join(train_img_dir, stem + ext)
        if os.path.exists(path):
            return path
    return None

MAX_DUP_FACTOR = 3
duplications = 0

for stem, cls_set in img_classes.items():
    if not cls_set or max_count == 0:
        continue

    min_class_count = min(class_counts[c] for c in cls_set)
    if min_class_count == 0:
        continue

    raw_factor = max_count / float(min_class_count)
    dup_factor = int(raw_factor)
    dup_factor = max(1, min(dup_factor, MAX_DUP_FACTOR))

    if dup_factor <= 1:
        continue

    img_path = find_image_for_stem(stem)
    lbl_path = os.path.join(train_lbl_dir, stem + ".txt")

    if img_path is None or not os.path.exists(lbl_path):
        continue

    img_root, img_ext = os.path.splitext(os.path.basename(img_path))

    for k in range(1, dup_factor):
        new_stem = f"{stem}_dup{k}"
        new_img_path = os.path.join(train_img_dir, new_stem + img_ext)
        new_lbl_path = os.path.join(train_lbl_dir, new_stem + ".txt")

        if os.path.exists(new_img_path) or os.path.exists(new_lbl_path):
            continue

        shutil.copy2(img_path, new_img_path)
        shutil.copy2(lbl_path, new_lbl_path)
        duplications += 1

print(f"  Oversampling done. Created {duplications} extra train image/label pair(s).")

ti_after = count_imgs(os.path.join(DATASET_DIR, "train", "images"))
print(f"  Train images after oversampling: {ti_after}")


HYP_YAML = "/content/hyp_minutiae.yaml"

hyp = dict(
    lr0=0.002,
    lrf=0.2,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,

    box=10.0,
    cls=0.7,
    dfl=1.5,

    hsv_h=0.0,
    hsv_s=0.1,
    hsv_v=0.1,
    degrees=0.0,
    translate=0.03,
    scale=0.1,
    shear=0.0,
    perspective=0.0,
    mosaic=0.2,
    mixup=0.0,
    copy_paste=0.0,
)

with open(HYP_YAML, "w") as f:
    yaml.safe_dump(hyp, f, sort_keys=False)

print("📝 Saved hyperparameters to", HYP_YAML)
print(open(HYP_YAML).read())


MODEL_YAML = "yolo11s_p2_p3sppf.yaml"

model_yaml_str = r"""
# YOLO11-s style model:
# - Standard SPPF at P5
# - Extra SPPF at P3 (more local context)
# - P2 (stride 4) detection head for tiny minutiae

nc: 1
depth_multiple: 1.0
width_multiple: 1.0

backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 3, 2]],      # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]],     # 1-P2/4
    [-1, 3, C2f, [128]],            # 2-P2/4 (C2f) -> P2
    [-1, 1, Conv, [256, 3, 2]],     # 3-P3/8
    [-1, 6, C2f, [256]],            # 4-P3/8 (C2f) -> P3
    [-1, 1, Conv, [512, 3, 2]],     # 5-P4/16
    [-1, 6, C2f, [512]],            # 6-P4/16 (C2f) -> P4
    [-1, 1, Conv, [1024, 3, 2]],    # 7-P5/32
    [-1, 3, C2f, [1024]],           # 8-P5/32 (C2f)
    [-1, 1, SPPF, [1024, 5]],       # 9-P5/32 (SPPF) -> P5_sppf
    [4, 1, SPPF, [256, 5]],         # 10-P3/8 (SPPF) -> P3_sppf
  ]

head:
  [
    # ---------- P5 -> P4 path ----------
    [9, 1, Conv, [512, 1, 1]],                     # 11
    [11, 1, nn.Upsample, [None, 2, "nearest"]],    # 12 up P5 -> /16
    [[12, 6], 1, Concat, [1]],                     # 13 cat( up_P5, P4 )
    [13, 3, C2f, [512]],                           # 14 refined P4

    # ---------- P4 -> P3 path (use P3_sppf = 10) ----------
    [14, 1, Conv, [256, 1, 1]],                    # 15
    [15, 1, nn.Upsample, [None, 2, "nearest"]],    # 16 up P4 -> /8
    [[16, 10], 1, Concat, [1]],                    # 17 cat( up_P4, P3_sppf )
    [17, 3, C2f, [256]],                           # 18 refined P3

    # ---------- P3 -> P2 path (small objects) ----------
    [18, 1, Conv, [128, 1, 1]],                    # 19
    [19, 1, nn.Upsample, [None, 2, "nearest"]],    # 20 up P3 -> /4
    [[20, 2], 1, Concat, [1]],                     # 21 cat( up_P3, P2 )
    [21, 3, C2f, [128]],                           # 22 refined P2 (small objects)

    # ---------- PAN bottom-up fusion including P2 ----------
    [22, 1, Conv, [256, 3, 2]],                    # 23 down P2 -> /8
    [[23, 18], 1, Concat, [1]],                    # 24 cat( down_P2, P3_refined )
    [24, 3, C2f, [256]],                           # 25 fused P3

    [25, 1, Conv, [512, 3, 2]],                    # 26 down -> /16
    [[26, 14], 1, Concat, [1]],                    # 27 cat( down_P3, P4_refined )
    [27, 3, C2f, [512]],                           # 28 fused P4

    [28, 1, Conv, [1024, 3, 2]],                   # 29 down -> /32
    [[29, 9], 1, Concat, [1]],                     # 30 cat( down_P4, P5_sppf )
    [30, 3, C2f, [1024]],                          # 31 fused P5

    # ---------- Detect on [P2, P3, P4, P5] ----------
    [[22, 25, 28, 31], 1, Detect, [nc]],           # 32
  ]
"""

with open(MODEL_YAML, "w") as f:
    f.write(model_yaml_str)

print(f"✅ YOLO11s P2 + P3-SPPF model YAML written to: {MODEL_YAML}")
print(open(MODEL_YAML).read())


device = 0 if torch.cuda.is_available() else "cpu"
IMGSZ = 640
EPOCHS = 200
BATCH  = 16 if torch.cuda.is_available() else 8

with open(HYP_YAML, "r") as f:
    hyp = yaml.safe_load(f)

model = YOLO(MODEL_YAML)

results = model.train(
    data=DATA_YAML,
    imgsz=IMGSZ,
    epochs=EPOCHS,
    batch=BATCH,
    workers=8 if torch.cuda.is_available() else 0,
    device=device,
    optimizer="adamw",
    cos_lr=True,
    patience=80,
    amp=True,
    cache=True,
    project="runs_minutiae_p3sppf",
    name="yolo11s_p2_p3sppf",
    **hyp
)

print("🏁 Training finished. Check runs_minutiae_p3sppf/yolo11s_p2_p3sppf*")


valid_dir = None
if os.path.isdir(valid_images):
    valid_dir = valid_images
elif os.path.isdir(val_images):
    valid_dir = val_images

if valid_dir is not None:
    best = sorted(glob.glob("runs_minutiae_p3sppf/yolo11s_p2_p3sppf*/weights/best.pt"))[-1]
    print("Using best weights:", best)

    model_trained = YOLO(best)

    _ = model_trained.predict(
        source=valid_dir,
        imgsz=IMGSZ,
        conf=0.25,
        iou=0.5,
        save=True
    )
    print("  Predictions saved in runs_minutiae_p3sppf/yolo11s_p2_p3sppf*/predict")
else:
    print("  No valid/val images folder found for prediction.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Torch: 2.9.0+cu126 | CUDA available: True
📤 Please select your dataset .zip …


Saving Fingerprint Minutiae.v5-augmented_v5.yolov11.zip to Fingerprint Minutiae.v5-augmented_v5.yolov11.zip
Uploaded: Fingerprint Minutiae.v5-augmented_v5.yolov11.zip
✅ Unzipped to: /content/dataset
📝 Patched /content/dataset/data.yaml
train: /content/dataset/train/images
val: /content/dataset/valid/images
test: ../test/images
nc: 3
names:
- burification
- center
- ending
roboflow:
  workspace: khubab-ahmad
  project: fingerprint-minutiae
  version: 5
  license: CC BY 4.0
  url: https://universe.roboflow.com/khubab-ahmad/fingerprint-minutiae/dataset/5

📊 Train images (before oversampling): 480
📊 Valid/Val images: 40
📂 /content/dataset/train/images -> 5 sample(s): ['101_1_jpg.rf.01c4b3021b799bc9268d94596fb89db9.jpg', '101_1_jpg.rf.6ba5ecf368ecb1837e920d6b82a543da.jpg', '101_1_jpg.rf.88793b9d9d9fa94a967c3441031c75ff.jpg', '101_1_jpg.rf.97af06b6b52429b57a0db16b08b0687d.jpg', '101_1_jpg.rf.a2c64aeb76e0950cee16d2a25e975e1c.jpg']
📂 /content/dataset/train/labels -> 5 sample(s): ['101_1_jpg.rf